# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

%matplotlib inline
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

In [2]:
simulation_name = "dinwoodie_base"

sim = Simulation(simulation_name, "DINWOODIE", "base.yaml")
sim.run()

# For convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 91.7%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.97,0.90,0.95,0.90,0.97,0.97,0.97,0.90,0.97,0.91,0.89,0.93,0.98,0.90,0.97,0.85,0.97,0.49,0.97,0.98,0.90,0.97,0.90,0.97,0.97,0.90,0.98,0.98,0.94,0.98,0.97,0.91,0.87,0.95,0.94,0.97,0.97,0.97,0.98,0.96,0.85,0.83,0.94,0.97,0.83,0.97,0.97,0.92,0.08,0.85,0.88,0.97,0.89,0.98,0.90,0.97,0.39,0.98,0.92,0.97,0.97,0.91,0.97,0.97,0.92,0.96,0.97,0.93,0.98,0.97,0.97,0.90,0.87,0.93,0.97,0.98,0.94,0.94,0.97,0.95


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,year,windfarm
0,2003,0.97
1,2004,0.93
2,2005,0.95
3,2006,0.94
4,2007,0.89
5,2008,0.90
6,2009,0.89
7,2010,0.89
8,2011,0.90
9,2012,0.90


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,month,windfarm
0,1,0.91
1,2,0.91
2,3,0.91
3,4,0.90
4,5,0.91
5,6,0.92
6,7,0.93
7,8,0.93
8,9,0.93
9,10,0.93


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

,year,month,windfarm
0,2003,1,0.98
1,2003,2,0.97
2,2003,3,0.97
3,2003,4,0.96
4,2003,5,0.97
5,2003,6,0.99
6,2003,7,0.98
7,2003,8,0.98
8,2003,9,0.98
9,2003,10,0.98


## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.44%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.42,0.46,0.43,0.46,0.46,0.46,0.43,0.46,0.44,0.42,0.44,0.47,0.44,0.46,0.40,0.46,0.23,0.46,0.47,0.43,0.46,0.43,0.46,0.47,0.43,0.47,0.47,0.45,0.47,0.46,0.43,0.41,0.45,0.44,0.46,0.46,0.46,0.47,0.45,0.40,0.40,0.45,0.46,0.39,0.46,0.47,0.44,0.03,0.40,0.42,0.46,0.42,0.47,0.42,0.46,0.18,0.47,0.44,0.47,0.46,0.43,0.46,0.46,0.44,0.46,0.46,0.44,0.47,0.46,0.46,0.43,0.41,0.45,0.46,0.47,0.45,0.45,0.46,0.46


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.43
2005,0.47
2006,0.44
2007,0.46
2008,0.47
2009,0.42
2010,0.38
2011,0.45


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.57
2,0.47
3,0.48
4,0.39
5,0.37
6,0.31
7,0.31
8,0.35
9,0.44


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.36
     3          0.37
     4          0.52
     5          0.34
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.43
     11         0.54
     12         0.52
2004 1          0.52
     2          0.49
     3          0.49
     4          0.36
     5          0.28
     6          0.36
     7          0.26
     8          0.39
     9          0.56
     10         0.55
     11         0.47
     12         0.48

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 76%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 99%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,year,Completion Rate
0,2003,0.97
1,2004,0.93
2,2005,0.97
3,2006,0.97
4,2007,1.00
5,2008,1.00
6,2009,0.99
7,2010,1.00
8,2011,1.02
9,2012,1.02


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,year,Completion Rate
0,1,0.83
1,2,1.06
2,3,1.01
3,4,1.07
4,5,1.01
5,6,1.04
6,7,0.99
7,8,1.03
8,9,0.96
9,10,0.97


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

,year,month,Completion Rate
0,2003,1,0.76
1,2003,2,1.18
2,2003,3,0.97
3,2003,4,1.04
4,2003,5,1.05
5,2003,6,0.98
6,2003,7,1.00
7,2003,8,0.90
8,2003,9,1.11
9,2003,10,0.98


## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $533,297.27/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","106,151,562.50"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,year,equipment_cost
0,2003,"11,254,093.75"
1,2004,"13,729,687.50"
2,2005,"11,922,875.00"
3,2006,"12,454,125.00"
4,2007,"14,324,437.50"
5,2008,"11,629,687.50"
6,2009,"12,905,687.50"
7,2010,"13,205,687.50"
8,2011,"13,574,437.50"
9,2012,"12,990,625.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1,"543,010.42","543,010.42","543,010.42",0.00,0.00
1,2,"495,250.00","495,250.00","495,250.00",0.00,0.00
2,3,"542,500.00","542,500.00","542,500.00",0.00,0.00
3,4,"525,000.00","525,000.00","525,000.00",0.00,0.00
4,5,"542,500.00","542,500.00","542,500.00",0.00,"36,028,125.00"
5,6,"525,000.00","525,000.00","525,000.00","2,660,000.00",0.00
6,7,"542,500.00","542,500.00","542,500.00",0.00,"34,237,500.00"
7,8,"542,500.00","542,500.00","542,500.00",0.00,0.00
8,9,"525,000.00","525,000.00","525,000.00",0.00,"35,214,062.50"
9,10,"542,500.00","542,500.00","542,500.00",0.00,"671,875.00"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

,year,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,2003,1,"54,760.42","54,760.42","54,760.42",0.00,0.00
1,2003,2,"49,000.00","49,000.00","49,000.00",0.00,0.00
2,2003,3,"54,250.00","54,250.00","54,250.00",0.00,0.00
3,2003,4,"52,500.00","52,500.00","52,500.00",0.00,0.00
4,2003,5,"54,250.00","54,250.00","54,250.00",0.00,"3,570,312.50"
5,2003,6,"52,500.00","52,500.00","52,500.00","266,000.00",0.00
6,2003,7,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
7,2003,8,"54,250.00","54,250.00","54,250.00",0.00,0.00
8,2003,9,"52,500.00","52,500.00","52,500.00",0.00,"2,750,000.00"
9,2003,10,"54,250.00","54,250.00","54,250.00",0.00,0.00


## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.97,0.97,0.96,0.91,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.66,0.66,0.65,0.10,0.56
2004,1.00,1.00,1.00,1.00,1.00
2005,1.00,1.00,1.00,1.00,1.00
2006,1.00,1.00,1.00,1.00,1.00
2007,1.00,1.00,1.00,1.00,1.00
2008,1.00,1.00,1.00,1.00,1.00
2009,1.00,1.00,1.00,1.00,1.00
2010,1.00,1.00,1.00,1.00,1.00
2011,1.00,1.00,1.00,1.00,1.00


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,year,total_labor_cost
0,2003,0.00
1,2004,0.00
2,2005,0.00
3,2006,0.00
4,2007,0.00
5,2008,0.00
6,2009,0.00
7,2010,0.00
8,2011,0.00
9,2012,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,month,total_labor_cost
0,1,0.00
1,2,0.00
2,3,0.00
3,4,0.00
4,5,0.00
5,6,0.00
6,7,0.00
7,8,0.00
8,9,0.00
9,10,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

,year,month,total_labor_cost
0,2003,1,0.00
1,2003,2,0.00
2,2003,3,0.00
3,2003,4,0.00
4,2003,5,0.00
5,2003,6,0.00
6,2003,7,0.00
7,2003,8,0.00
8,2003,9,0.00
9,2003,10,0.00


## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,reason,total_cost
0,Maintenance,"1,659,109.38"
1,Repair,"24,702,888.02"
2,Mobilization,"15,000,000.00"
3,Weather Delay,"16,221,229.17"
4,No Requests,"7,853,776.04"
5,Not in Shift,"31,953,601.56"
6,NaN,"29,173,375.00"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,reason,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
0,Maintenance,0.00,0.00,0.00,"1,659,109.38","1,659,109.38"
1,Repair,0.00,0.00,0.00,"24,702,888.02","24,702,888.02"
2,Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
3,Weather Delay,0.00,0.00,0.00,"16,221,229.17","16,221,229.17"
4,No Requests,0.00,0.00,0.00,"7,853,776.04","7,853,776.04"
5,Not in Shift,0.00,0.00,0.00,"31,953,601.56","31,953,601.56"
6,NaN,0.00,0.00,0.00,"29,173,375.00","29,173,375.00"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

,year,reason,total_cost
0,2003,Maintenance,"35,729.17"
1,2003,Repair,"1,673,171.88"
2,2003,Mobilization,"1,500,000.00"
3,2003,Weather Delay,"605,111.98"
4,2003,No Requests,"4,129,088.54"
5,2003,Not in Shift,"1,748,966.15"
6,2003,NaN,"1,473,125.00"
7,2004,Maintenance,"149,187.50"
8,2004,Repair,"2,881,257.81"
9,2004,Mobilization,"1,500,000.00"


In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

,month,reason,total_cost
0,1,Maintenance,"126,601.56"
1,1,Repair,"445,447.92"
2,1,Mobilization,0.00
3,1,Weather Delay,"148,221.35"
4,1,No Requests,"70,072.92"
5,1,Not in Shift,"183,002.60"
6,1,NaN,"626,500.00"
7,2,Maintenance,"124,377.60"
8,2,Repair,"438,138.02"
9,2,Mobilization,0.00


In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

,year,month,reason,total_cost
0,2003,1,Maintenance,0.00
1,2003,1,Repair,"38,445.31"
2,2003,1,Mobilization,0.00
3,2003,1,Weather Delay,"13,763.02"
4,2003,1,No Requests,"70,072.92"
5,2003,1,Not in Shift,"2,460.94"
6,2003,1,NaN,"37,625.00"
7,2003,2,Maintenance,0.00
8,2003,2,Repair,"58,005.21"
9,2003,2,Mobilization,0.00


## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,component,total_cost
0,cable,0.00
1,drive_train,0.00
2,electrical_system,0.00
3,electronic_control,0.00
4,gearbox,0.00
5,generator,"137,589,036.46"
6,hydraulic_system,0.00
7,mechanical_brake,0.00
8,rotor_blades,0.00
9,rotor_hub,0.00


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

,component,action,materials_cost,total_labor_cost,equipment_cost,total_cost
0,generator,delay,0,0.00,"77,346,674.48","77,346,674.48"
1,generator,maintenance,0,0.00,"1,659,109.38","1,659,109.38"
2,generator,repair,0,0.00,"24,702,888.02","24,702,888.02"


In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,year,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,cable,0.00,0.00,0.00,0.00
1,2003,drive_train,0.00,0.00,0.00,0.00
2,2003,electrical_system,0.00,0.00,0.00,0.00
3,2003,electronic_control,0.00,0.00,0.00,0.00
4,2003,gearbox,0.00,0.00,0.00,0.00
5,2003,generator,"1,824,500.00",0.00,"5,623,473.96","7,447,973.96"
6,2003,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,rotor_blades,0.00,0.00,0.00,0.00
9,2003,rotor_hub,0.00,0.00,0.00,0.00


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,1,cable,0.00,0.00,0.00,0.00
1,1,drive_train,0.00,0.00,0.00,0.00
2,1,electrical_system,0.00,0.00,0.00,0.00
3,1,electronic_control,0.00,0.00,0.00,0.00
4,1,gearbox,0.00,0.00,0.00,0.00
5,1,generator,"931,000.00",0.00,"1,557,427.08","2,488,427.08"
6,1,hydraulic_system,0.00,0.00,0.00,0.00
7,1,mechanical_brake,0.00,0.00,0.00,0.00
8,1,rotor_blades,0.00,0.00,0.00,0.00
9,1,rotor_hub,0.00,0.00,0.00,0.00


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

,year,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,1,cable,0.00,0.00,0.00,0.00
1,2003,1,drive_train,0.00,0.00,0.00,0.00
2,2003,1,electrical_system,0.00,0.00,0.00,0.00
3,2003,1,electronic_control,0.00,0.00,0.00,0.00
4,2003,1,gearbox,0.00,0.00,0.00,0.00
5,2003,1,generator,"32,500.00",0.00,"92,677.08","125,177.08"
6,2003,1,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,1,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,1,rotor_blades,0.00,0.00,0.00,0.00
9,2003,1,rotor_hub,0.00,0.00,0.00,0.00


## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,year,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
1,2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2,2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
3,2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
4,2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
5,2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
6,2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
7,2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
8,2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
9,2012,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"7,069,457.00","60,635.00","52,681.50",468.00
major repair,"63,701.34","1,292.50",980.00,18.00
major replacement,"237,238.80","13,752.50","237,238.80",57.00
manual reset,"628,332.12","132,304.75","12,633.75","5,369.00"
medium repair,"14,118.72","10,837.25","6,779.50",195.00
minor repair,"107,001.00","52,286.50","5,537.25","2,186.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,193,383,472.50"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,193,383,472.50","122,016,054.50","110,994,133.00","120,136,846.00","113,670,080.00","121,587,375.50","121,733,705.00","121,799,692.00","113,681,243.50","121,630,165.50","115,038,122.50","110,754,227.50","116,364,760.50","122,457,941.00","114,517,598.50","121,541,953.00","104,660,305.50","121,514,720.00","61,308,115.50","121,685,366.50","122,599,117.50","113,531,508.50","122,109,803.00","112,498,571.00","122,215,866.00","122,311,163.00","114,308,693.50","122,844,236.00","122,649,474.50","117,473,885.50","122,603,332.00","122,286,035.50","114,409,032.00","108,684,703.00","119,290,070.50","115,699,536.50","121,973,732.50","121,645,202.50","121,729,485.50","122,833,532.00","119,297,792.00","104,764,398.00","104,241,640.50","117,928,409.50","121,636,218.00","102,180,308.00","121,423,522.50","122,385,019.00","115,958,073.50","8,138,013.00","104,851,958.50","110,646,059.50","121,128,824.00","110,784,074.00","122,794,273.00","110,722,750.00","122,023,384.00","47,711,095.50","122,754,620.50","115,185,157.00","122,375,320.00","122,104,754.50","114,194,507.00","121,805,339.50","122,002,198.50","115,453,698.00","120,852,767.50","121,582,164.50","115,858,698.00","122,923,070.50","121,565,555.50","121,407,347.50","112,788,737.00","109,140,219.00","117,534,536.00","122,282,346.50","122,345,556.50","119,302,958.50","118,684,993.00","121,585,665.50","120,248,063.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"855,742,495.00"
2004,"915,172,641.50"
2005,"978,577,112.50"
2006,"927,792,482.50"
2007,"957,421,048.50"
2008,"984,222,767.50"
2009,"886,078,009.00"
2010,"798,753,155.00"
2011,"956,150,848.50"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,024,489,586.00"
2,"772,729,025.00"
3,"848,710,944.00"
4,"667,809,736.00"
5,"651,945,802.50"
6,"531,390,598.50"
7,"561,336,354.50"
8,"619,873,129.00"
9,"764,602,248.50"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     110,236,786.00
     2      57,711,864.00
     3      66,139,682.50
     4      90,222,303.50
     5      60,961,353.50
     6      53,566,087.00
     7      53,356,025.00
     8      53,144,416.00
     9      47,232,595.00
     10     76,641,699.50
     11     92,859,533.50
     12     93,670,149.50
2004 1      92,845,411.50
     2      81,288,084.00
     3      86,603,887.50
     4      62,588,281.00
     5      49,232,269.50
     6      62,221,401.50
     7      45,993,451.00
     8      70,455,003.00
     9      97,343,603.50
     10     98,351,004.00
     11     81,923,584.50
     12     86,326,660.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [54]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

In [55]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"27,620,287.62"
Nominal LCOE ($/kW),-0.00
Real LCOE ($/kW),-0.00
After Tax IRR (%),13.63


In [56]:
sim.env.cleanup_log_files(log_only=False)